In [117]:
import time
import numpy as np
import pandas as pd
import requests

import csv
import os

import json
from pandas.io.json import json_normalize

In [40]:
import _local_settings

In [41]:
# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)


In [207]:


app_file = 'response/applist.csv'
processed_app_file ='response/processed_applist.csv'

success_file = 'response/success.json'
failure_file = 'response/failure.json'

In [208]:
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.

    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request

    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """        
    try:
        response = requests.get(url=url, params=parameters)
    except requests.exceptions.Timeout:
        for i in range(5, 0, -1):
            print('\rTimeout. Waiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' ' * 10)
        return get_request(url, parameters)
    except requests.exceptions.TooManyRedirects:
        print('TooManyRedirects')
    except requests.exceptions.RequestException as e:
    # catastrophic error. bail.
        raise SystemExit(e)

    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again
        print('No response, waiting 10 seconds...')
        time.sleep(10)
        print('Retrying.')
        return get_request(url, parameters)

def write_to_file(d):
    for app in d['applist']['apps']:
        appid = app['appid']
        with open(app_file, 'a') as applist:
            json.dump(appid, applist)
            applist.write(",")
    
    with open(app_file, 'a') as applist:
        applist.seek(0, os.SEEK_END) # Move to last 
        applist.seek(applist.tell() - 1, os.SEEK_SET) # back One character
        applist.truncate() # Delete the last comma ","
        

In [224]:
def get_unprocessed_applist():
    '''
    Gets the list of all steam app ids, then gets a list of app ids that have 
    already been processed and produces a series of just the unprocessed app ids
    '''
    all_apps_file = pd.read_csv(app_file, index_col=False, header=0)
    all_apps_series = all_apps_file.transpose().reset_index()
    
    processed_apps_exist = False
    
    try:
        processed_apps = pd.read_csv(processed_app_file, index_col=False, header=0)
        processed_apps_series = processed_apps.transpose().reset_index()
        processed_apps_exist = True
    except:
      print(processed_app_file, " is empty")
    
    if processed_apps_exist:
        return pd.concat([all_apps_series,processed_apps_series]).drop_duplicates(keep=False)
    else:
        return all_apps_series




In [335]:
def get_app_info(apps, limit=200):
# has logic to deal with 200 requests/5 min limits
    with open(success_file, 'a') as successes:
        successes.write('{"responses":[')
    with open(failure_file, 'a') as failures:
        failures.write('{"responses":[')
        
    count = 0
    
    for app in apps:
        appid = app
        print(appid)
        count+= 1
        app_info = get_request("http://store.steampowered.com/api/appdetails?appids={APP_ID}".format(APP_ID=appid))
        print(type(app_info))
        df = pd.DataFrame.from_dict(app_info)
        print(type(df))
        print(df)
        
        
        time.sleep(1.6)  # should keep us below request limit

        if app_info[str(appid)]['success']:
            
#             df.to_json(success_file, orient='index)
            with open(success_file, 'a') as successes:
                json.dump(app_info, successes)  # write api resp to .JSON file
#               json.dump(app_info[str(appid)]['data'], successes)  # write api resp to .JSON file
                successes.write(",") # add comma for JSON array element
        else:            
#             df.to_json(failure_file, orient='index')
            with open(failure_file, 'a') as failures:

                json.dump(app_info, failures)  # write api resp to .JSON file
                failures.write(",") # add comma for JSON array element
        
        with open(processed_app_file, 'a') as processed_applist:
            json.dump(appid, processed_applist) 
            processed_applist.write(",")
        if count >= limit:
            print("reached " + str(limit) + " requests, stopping")
            break
    
    with open(success_file, 'a') as successes:
        successes.seek(0, os.SEEK_END) # Move to last 
        successes.seek(successes.tell() - 1, os.SEEK_SET) # back One character
        successes.truncate() # Delete the last comma ","
        successes.write(']}')
    with open(failure_file, 'a') as failures:
        failures.seek(0, os.SEEK_END) # Move to last 
        failures.seek(failures.tell() - 1, os.SEEK_SET) # back One character
        failures.truncate() # Delete the last comma ","
        failures.write(']}')
    
    with open(processed_app_file, 'a') as processed_applist:
        processed_applist.seek(0, os.SEEK_END) # Move to last 
        processed_applist.seek(processed_applist.tell() - 1, os.SEEK_SET) # back One character
        processed_applist.truncate() # Delete the last comma ",

    print("I'\'m done writing your successes to file:" + success_file)
    print("I'\'m done writing your failures to file:" + failure_file)




In [212]:
ALL_APPS_URL = "http://api.steampowered.com/ISteamApps/GetAppList/v0002/?key={STEAMKEY}&format=json".format(STEAMKEY=_local_settings.STEAMKEY)

all_apps = get_request(ALL_APPS_URL)
write_to_file(all_apps)

In [277]:
unprocessed_appids = get_unprocessed_applist()
print(unprocessed_appids.head())



response/processed_applist.csv  is empty
     index
0   216938
1   660010
2   660130
3  1095230
4  1095260


In [234]:
count=0
d = all_apps
for app in d['applist']['apps']:
    count+=1
print(count)

99933


In [336]:
get_app_info(unprocessed_appids['index'], limit=5)
# 216938
# 660010
# 660130
# 1095230
# 1095260
# 1095270

216938
<class 'dict'>
<class 'pandas.core.frame.DataFrame'>
         216938
success   False
660010
<class 'dict'>
<class 'pandas.core.frame.DataFrame'>
         660010
success   False
660130
<class 'dict'>
<class 'pandas.core.frame.DataFrame'>
         660130
success   False
1095230
<class 'dict'>
<class 'pandas.core.frame.DataFrame'>
                                                   1095230
data     {'type': 'game', 'name': 'Fish man avoiding fi...
success                                               True
1095260
<class 'dict'>
<class 'pandas.core.frame.DataFrame'>
                                                   1095260
data     {'type': 'game', 'name': 'Airtight City 密闭之城',...
success                                               True
reached 5 requests, stopping
I''m done writing your successes to file:response/success.json
I''m done writing your failures to file:response/failure.json


## test reading output files

In [372]:
s = pd.read_json(success_file, orient='records')
s


,responses
0,"{'1095230': {'success': True, 'data': {'type':..."
1,"{'1095260': {'success': True, 'data': {'type':..."


In [444]:
df = pd.read_json(failure_file, orient='index')
print(df.T)
f = df.T.to_dict()
print(f)
def convert_to_df(d):
    df = pd.DataFrame.from_dict(d, orient='columns')
    df.reset_index(level=0, inplace=True)
    return df
x = convert_to_df(f)

z = {'216938': {'success': False}},{'660010': {'success': False}},{'660130': {'success': False}}
abc = convert_to_df(z)
abc

#todo: need to convert each thing from dict to just the k/v pair

                        responses
0  {'216938': {'success': False}}
1  {'660010': {'success': False}}
2  {'660130': {'success': False}}
{'responses': {0: {'216938': {'success': False}}, 1: {'660010': {'success': False}}, 2: {'660130': {'success': False}}}}


,index,216938,660010,660130
0,0,{'success': False},NaN,NaN
1,1,NaN,{'success': False},NaN
2,2,NaN,NaN,{'success': False}


In [395]:
steam_data = pd.read_json(success_file)
steam_data

f = pd.DataFrame([steam_data['responses']])
f


def convert_to_df(d):
    df = pd.DataFrame.from_dict(d, orient='columns')
    df.reset_index(level=0, inplace=True)
    return df

# steam_data['responses'].head()

x = convert_to_df(steam_data['responses'].to_dict())
x


# from ast import literal_eval
# def try_literal_eval(s):
#     try:
#         return literal_eval(s)
#     except ValueError:
#         return s

# steam_data['responses'] = steam_data.responses.apply(try_literal_eval)
# app_df = convert_to_df(steam_data)
# appdf


# result = steam_data['responses'].to_json(orient="index")
# parsed = json.loads(result)
# test = json.dumps(parsed, indent=4)  
# print(test)
# norm = pd.json_normalize(test)



,index,0,1
0,1095230,"{'success': True, 'data': {'type': 'game', 'na...",NaN
1,1095260,NaN,"{'success': True, 'data': {'type': 'game', 'na..."
